In [1]:
#imports and setup
import numpy as np
import cv2 as cv
import os

In [2]:
'''loading video names into a list for easier access with absolute path,
video names can be access by calling the index of the video_lst and will contain 
the current absolute path, asumes that the video folder is in the same folder as this file'''

video_path = "../videos"
full_path = os.path.abspath(video_path)

video_lst = []
v_dir = os.listdir(full_path)
for name in v_dir:
    video_lst.append(full_path+'/'+name)
print(video_lst[0]) 

/Users/daryashitova/Documents/MA_AI/WS22:23/Computer Vision/project/videos/video_026.mp4


In [3]:
#loading video into memory
cap = cv.VideoCapture(video_lst[0])

In [4]:
# import the necessary packages
from scipy.spatial import distance as dist
from collections import OrderedDict
import numpy as np
class CentroidTracker():
	def __init__(self, maxDisappeared=50):
		# initialize the next unique object ID along with two ordered
		# dictionaries used to keep track of mapping a given object
		# ID to its centroid and number of consecutive frames it has
		# been marked as "disappeared", respectively
		self.nextObjectID = 0
		self.objects = OrderedDict()
		self.disappeared = OrderedDict()
		# store the number of maximum consecutive frames a given
		# object is allowed to be marked as "disappeared" until we
		# need to deregister the object from tracking
		self.maxDisappeared = maxDisappeared

	def register(self, centroid):
		# when registering an object we use the next available object
		# ID to store the centroid
		self.objects[self.nextObjectID] = centroid
		self.disappeared[self.nextObjectID] = 0
		self.nextObjectID += 1

	def deregister(self, objectID):
		# to deregister an object ID we delete the object ID from
		# both of our respective dictionaries
		del self.objects[objectID]
		del self.disappeared[objectID]

	def update(self, rects):
		# check to see if the list of input bounding box rectangles
		# is empty
		if len(rects) == 0:
			# loop over any existing tracked objects and mark them
			# as disappeared
			for objectID in list(self.disappeared.keys()):
				self.disappeared[objectID] += 1
				# if we have reached a maximum number of consecutive
				# frames where a given object has been marked as
				# missing, deregister it
				if self.disappeared[objectID] > self.maxDisappeared:
					self.deregister(objectID)
			# return early as there are no centroids or tracking info
			# to update
			return self.objects
		# initialize an array of input centroids for the current frame
		inputCentroids = np.zeros((len(rects), 2), dtype="int")
		# loop over the bounding box rectangles
		for (i, (startX, startY, endX, endY)) in enumerate(rects):
			# use the bounding box coordinates to derive the centroid
			cX = (startX + startX + endX) // 2.0
			cY = (startY + startY + endY) // 2.0
			inputCentroids[i] = (cX, cY)
		# if we are currently not tracking any objects take the input
		# centroids and register each of them
		if len(self.objects) == 0:
			for i in range(0, len(inputCentroids)):
				self.register(inputCentroids[i])
		# otherwise, are are currently tracking objects so we need to
		# try to match the input centroids to existing object
		# centroids
		else:
			# grab the set of object IDs and corresponding centroids
			objectIDs = list(self.objects.keys())
			objectCentroids = list(self.objects.values())
			# compute the distance between each pair of object
			# centroids and input centroids, respectively -- our
			# goal will be to match an input centroid to an existing
			# object centroid
			D = dist.cdist(np.array(objectCentroids), inputCentroids)
			# in order to perform this matching we must (1) find the
			# smallest value in each row and then (2) sort the row
			# indexes based on their minimum values so that the row
			# with the smallest value is at the *front* of the index
			# list
			rows = D.min(axis=1).argsort()
			# next, we perform a similar process on the columns by
			# finding the smallest value in each column and then
			# sorting using the previously computed row index list
			cols = D.argmin(axis=1)[rows]
			# in order to determine if we need to update, register,
			# or deregister an object we need to keep track of which
			# of the rows and column indexes we have already examined
			usedRows = set()
			usedCols = set()
			# loop over the combination of the (row, column) index
			# tuples
			for (row, col) in zip(rows, cols):
				# if we have already examined either the row or
				# column value before, ignore it
				# val
				if row in usedRows or col in usedCols:
					continue
				# otherwise, grab the object ID for the current row,
				# set its new centroid, and reset the disappeared
				# counter
				objectID = objectIDs[row]
				self.objects[objectID] = inputCentroids[col]
				self.disappeared[objectID] = 0
				# indicate that we have examined each of the row and
				# column indexes, respectively
				usedRows.add(row)
				usedCols.add(col)
			# compute both the row and column index we have NOT yet
			# examined
			unusedRows = set(range(0, D.shape[0])).difference(usedRows)
			unusedCols = set(range(0, D.shape[1])).difference(usedCols)
			# in the event that the number of object centroids is
			# equal or greater than the number of input centroids
			# we need to check and see if some of these objects have
			# potentially disappeared
			if D.shape[0] >= D.shape[1]:
				# loop over the unused row indexes
				for row in unusedRows:
					# grab the object ID for the corresponding row
					# index and increment the disappeared counter
					objectID = objectIDs[row]
					self.disappeared[objectID] += 1
					# check to see if the number of consecutive
					# frames the object has been marked "disappeared"
					# for warrants deregistering the object
					if self.disappeared[objectID] > self.maxDisappeared:
						self.deregister(objectID)
			# otherwise, if the number of input centroids is greater
			# than the number of existing object centroids we need to
			# register each new input centroid as a trackable object
			else:
				for col in unusedCols:
					self.register(inputCentroids[col])
		# return the set of trackable objects
		return self.objects

/Users/daryashitova/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
# Utility Functions
# Compute the frame difference
def frame_diff(prev_frame, cur_frame, next_frame):
    diff_frames1 = cv.absdiff(next_frame, cur_frame)
    # Absolute difference between current frame and previous frame
    diff_frames2 = cv.absdiff(cur_frame, prev_frame)
    # Return the result of bitwise 'AND' between the above two resultant images
    #gives better result than simple substraction
    return cv.bitwise_and(diff_frames1, diff_frames2)

def get_frame(cap):
    ret, frame = cap.read()
    # Resize the image not needed
    '''frame = cv.resize(frame, None, fx=scaling_factor,
        fy=scaling_factor, interpolation=cv.INTER_AREA)'''
    return frame

#finds and draws contours over pixels if greater than threshold
def construct_contours(frame, contour_threshold):
    frame = cv.dilate(frame, (1,1), iterations=1)
    contours, hierarchy = cv.findContours(frame, cv.RETR_TREE, 
                                           cv.CHAIN_APPROX_SIMPLE)
    '''cv.drawContours(frame_th, contours=contours, contourIdx=-1, 
                     color=(0, 255, 0), thickness=2, lineType=cv.LINE_AA)'''
    detections=[]
    for contour in contours:
      if cv.contourArea(contour) < contour_threshold:
        continue
      (x, y, w, h) = cv.boundingRect(contour)
      #cv.rectangle(frame, pt1=(x, y), pt2=(x + w, y + h), color=(0, 255, 0), thickness=2)
      detections.append([x,y,w,h])
    
    return detections

    #applying blur will possibly extend with other option for preperation
def prepare_frames(frames,kernel):
    result = []
    for frame in frames:
        result.append(cv.GaussianBlur(frame, kernel, 0))
    return result[0], result[1], result[2]

In [6]:
'''Frame differencing code based on:
https://github.com/infoaryan/OPENCV-PYTHON-Zero-to-One-Course-Resources/blob/master/Video%2031%20-%20Frame%20Differencing/frame_differencing.py
Expanded with: blur, dilation and morphological noise reduction and contour finding

Threshold for noise reduction and contour finding is adapted dynamically and will be 
calculated in every frame
'''

#video to capture
video = sorted(video_lst)[0]

kernel = np.ones((7,7)) #kerne for dilation, erosion
kernel_blur = (3,3)
kernel_morph = cv.getStructuringElement(cv.MORPH_ELLIPSE, (7, 7)) #kernel for morphology
threshold = 0 #when to detect difference
VALUE = 255 #which value to assign to difference
frame_counter = 0
power = 1 #magnifying factor for adaptive threshold

#threshold to calculate
threshold_method = cv.THRESH_BINARY
tracker = CentroidTracker()
centerpoints = []

# Compute the frame difference
def frame_diff(prev_frame, cur_frame, next_frame):
    diff_frames1 = cv.absdiff(next_frame, cur_frame)
    # Absolute difference between current frame and previous frame
    diff_frames2 = cv.absdiff(cur_frame, prev_frame)
    # Return the result of bitwise 'AND' between the above two resultant images
    #gives better result than simple substraction
    return cv.bitwise_and(diff_frames1, diff_frames2)

def get_frame(cap):
    ret, frame = cap.read()
    # Resize the image if convolution made the tracking area smaller
    '''frame = cv.resize(frame, None, fx=1,
        fy=1, interpolation=cv.INTER_AREA)'''
    return frame

#finds and draws contours over pixels if greater than threshold
def construct_contours(frame, contour_threshold):
    #frame = cv.dilate(frame, (1,1), iterations=1)
    contours, hierarchy = cv.findContours(frame, cv.RETR_TREE, 
                                           cv.CHAIN_APPROX_SIMPLE)
    '''cv.drawContours(frame_th, contours=contours, contourIdx=-1, 
                     color=(0, 255, 0), thickness=2, lineType=cv.LINE_AA)'''
    detections=[]
    for contour in contours:
      if cv.contourArea(contour) < contour_threshold:
        continue
      (x, y, w, h) = cv.boundingRect(contour)
      #cv.rectangle(frame, pt1=(x, y), pt2=(x + w, y + h), color=(0, 255, 0), thickness=2)
      detections.append([x,y,w,h])
    
    return detections


cap =cv.VideoCapture(video)
prev_frame = get_frame(cap)
cur_frame = get_frame(cap)
next_frame = get_frame(cap)

tracks = {}

'''Check if directory exists, if not, create it'''
import os

# You should change 'test' to your preferred folder.
dir = ("outputs")
check = os.path.isdir(dir)
# If folder doesn't exist, then create it.
if not check:
    os.makedirs(dir)

file_name = video.split("/")[-1].split(".")
file_name[0] = file_name[0]+"_out"
output_name = ".".join(file_name)
full_output_path = f"{dir}/{output_name}"

result = cv.VideoWriter(full_output_path, cv.VideoWriter_fourcc(*'XVID'), 20, (640, 512))


#applying blur will possibly extend with other option for preperation
def prepare_frames(frames,kernel):
    result = []
    for frame in frames:
        result.append(cv.GaussianBlur(frame, kernel, 0))
    return result[0], result[1], result[2]

# Iterating over all frames and applying difference and dilation
while True:
    frames = [prev_frame,cur_frame,next_frame]
    frame_counter += 1
    #applying preprocessing
    prev_frame, cur_frame, next_frame = prepare_frames(frames, kernel_blur)
    
    #cv.imshow("Blur", cur_frame)
    #calculating difference
    frame_difference = frame_diff(prev_frame, cur_frame, next_frame)
    #cv.imshow("Difference", frame_difference)
   
    #applying dilation
    frame_difference = cv.erode(frame_difference, (1,1))
    frame_difference = cv.dilate(frame_difference, kernel)
    #cv.imshow("Dilation", frame_difference)
    
    #applying morphological noise reduction
    frame_difference = cv.morphologyEx(frame_difference, cv.MORPH_OPEN, kernel_morph)
    #cv.imshow("Morph", frame_difference)
   
    #grayscale conversion
    frame_difference = cv.cvtColor(frame_difference, cv.COLOR_BGR2GRAY)

    #applying thresholds
    ret, frame_th = cv.threshold(frame_difference, threshold, VALUE, threshold_method)
    #cv.imshow("Threshold", frame_difference)
    
    #finding and drawing contours on given frame with given threshold
    detections = construct_contours(frame_th, threshold)
    #cv.imshow("Contours", detections)
    #cv.imshow("Difference", frame_difference)
    #cv.imshow("After Threshold", frame_th)
    #frame_difference_blured = cv.medianBlur(frame_difference,7)
    
    frame_difference_blured = cv.blur(frame_difference,(10,10)) # needed for adapting threshold
    
    '''Dynamic threshold adaption'''
    threshold_new = np.quantile(frame_difference_blured, 0.9999)
    frame_skip = 1#how many frames to skip
    
    if frame_counter == 1:
        threshold = threshold_new
    if frame_counter % frame_skip == 0: 
        if threshold_new > 0.995**power * threshold:
            threshold = threshold_new
            power = 1
        if threshold_new <= 0.995**power * threshold:
            power += 1
    

    if frame_counter > 10:
        boxes_ids = tracker.update(detections)
        for (objectID, centroid) in boxes_ids.items():
            if objectID in tracks:
                tracks[objectID].append(centroid)
            else:
                tracks[objectID] = [centroid]
            
            centerpoints.append((centroid[0], centroid[1]))
            text = f"ID {objectID}"

            cv.putText(cur_frame, text, (centroid[0], centroid[1]-15),
                cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0, 255, 0), 2)
            #cv.putText(cur_frame,  str(id),(x,y-15), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)
            cv.rectangle(cur_frame, (centroid[0]-10, centroid[1]-10), (centroid[0]+10, centroid[1]+10), (0,255,0), 2)

            
        for key in tracks:
            for i, point in enumerate(tracks[key][1:]):
                cv.line(cur_frame, tracks[key][i], point, (0,255,0), 3) 

    result.write(cur_frame)
    # Update the variables
    prev_frame = cur_frame
    cur_frame = next_frame
    # next_frame = get_frame(cap)
    ret, next_frame = cap.read()
    
    if not ret:
        break

    if cv.waitKey(25) & 0xFF == ord('q'):
        break    
cap.release()
result.release()
cv.waitKey(1)

cv.destroyAllWindows()
for i in range (1,5):
    cv.waitKey(1)


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
